# [Ford-Fulkerson Algorithm](https://en.wikipedia.org/wiki/Ford–Fulkerson_algorithm)


The Ford–Fulkerson method 혹은 Ford–Fulkerson algorithm (FFA) 는 flow network 에서 최대 유량을 계산하는 그리디 알고리즘 입니다.


residual graph에서 augmenting paths를 찾는 접근법이 완전히 지정되지 않았으며, 혹은 어떻게 구현하냐 에따라 실행 시간이 다르기 때문에 알고리즘 대신 'Method' 라고 부르기도 합니다.



1956년 L. R. Ford Jr.와 D. R. Fulkerson.에 의해 출판되었습니다.


## Flow network 정의


- **Flow network** (유량 네트워크): edges에 source (S) 와  sink (T) 를 가진 정점들의 네트워크로 각각의 간선의 값은 비용이 아닌 **용량을 갖는 방향성 그래프** 입니다.
 


- **$c(u,v)$** : 정점 $u$ 에서 $v$ 로 갈 수 있는 / 남은 간성의 용량 ( capacity )


- **$f(u,v)$** : 정점 $u$ 에서 $v$ 로 실제 흐르는 / 사용하고 있는 유량 ( flow )


- **Source** : 시작점


- **Sink** : 도착점




- **Augmenting Path** : flow network에서 Source에서 Sink로 가는 경로


- **Residual Graph** : 추가로 흐름이 가능한 flow network Graph


- **Residual Capacity** : 잔여 용량. 이미 흐르고 있는 유량 외에 추가로 유량을 보낼 수 있는 여유 용량



## Flow network 속성


$f(e) \le c(e)$ 
- **용량 제한 속성** : 각 간선의 유량은 그 간선의 용량을 넘을 수 없다.



$\sum_{(v, u) \in E} f((v, u)) = \sum_{(u, v) \in E} f((u, v))$ 
- **유량의 대칭성** : $u$ 에서 $v$ 로 유량이 흐르면, $v$ 에서 $u$ 로 음수의 유량이 흐르는것 과 동일



$\sum_{(s, u) \in E} f((s, u)) = \sum_{(u, t) \in E} f((u, t))$ 
- **유량의 보존** : 각 정점에 들어오는 유량과 나가는 유량은 같음




## Ford-Fulkerson Algorithm 정의


**입력** : Given a Network $G=(V,E)$, 용량 $c$, 소스 노드 $s$, 싱크 노드 $t$


**출력** : 소스 $s$에서 싱크 $t$로 흐를수 있는 최대 유량


## Ford-Fulkerson Algorithm 동작 과정

1. 모든 간선의 유량을 0으로 초기화 


2. 더 이상 증가 경로가 존재하지 않을 때까지 증가 경로 탐색하여 보낼 수 있는 최대 유량 보내기 


3. 잔여 용량 계산 및 유량의 대칭성을 이용하여 유량 상쇄하여 최대 유량 계산



## 유량 상쇄의 필요성

- **증강 경로를 어떻게 선택하는냐에 따라 최대 유량을 찾을 수 없는 경우가 있으므로 유량 상쇄가 필요합니다.**


- 새 유량을 보내는 것과 기존의 유량을 상쇄하는 것은 같은 연산입니다.

$(a,b) = 1$ 인 상태에서

$(b,a)$ 간선은 없으므로, $c(b,a) = 0$, 유량의 대칭성에 의하여 $f(b,a) = -1$



- 존재하지 않는 간선으로 유량을 흘려보낸다
    - 흘러오는 유량을 줄이는 것은 상대에게 유량을 보내주는 것과 같은 효과
    - 즉, a와 b는 서로 상대에게 유량을 보내 주는 것이 의미가 없어짐
    
    

## Src - Ford-Fulkerson Algorithm - BFS

**graph1** 

![graph1](img/g1.png) 

---
**graph2**

![graph2](img/g2.png)



In [1]:
# Ford-Fulkerson algorith in Python

from collections import defaultdict

# 인접 행렬 표현을 사용하는 방향 그래프를 나타냅니다.
class Graph:

    def __init__(self, graph):
        self.graph = graph
        self.row = len(graph)


    # BFS 탐색 알고리즘을 사용하여 Source s 에서 Sink t 로 가는 경로를 찾습니다. = Edmonds-Karp
    
    def searching_algo_BFS(self, s, t, parent):

        visited = [False] * (self.row) # 각 노드가 방문된 정보를 리스트 자료형으로 표현(1차원 리스트) 아직 방문하지 않은 상태
        
        queue = [] # 큐 생성

        queue.append(s)   # enqueue
        visited[s] = True # 방문 처리

        
        # BFS Loop
        while queue:
            u = queue.pop(0)

            for ind, val in enumerate(self.graph[u]):
                
                if (visited[ind] == False) and (val > 0):
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u
                    
                    if visited[t] : return True
        
        # Source에서 Sink로 가는 경로를 찾지 못하면 False 반환 합니다. 
        return False         

    # ford-fulkerson algorithm을 이용하여 최대 유량 반환
    
    def ford_fulkerson(self, source, sink):
        
        parent = [-1] * (self.row)
        max_flow = 0 # 초기 0 
        
        # Source s 에서 Sink t로 가는 경로가 있으면 +해서 최대 유량 구하기
        while self.searching_algo_BFS(source, sink, parent):
            
            # 증가 경로 상의 간선 중 잔여 용량이 가장 낮은 것을 찾습니다.
            path_flow = float("Inf")
            s = sink
            
            while(s != source):
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            # path 횟수 증가
            max_flow += path_flow
            
            # 잔여 용량 업데이트 
            v = sink          
            while(v != source):
                
                u = parent[v]
                self.graph[u][v] -= path_flow # 역방향에는 -로 흘려줍니다.
                self.graph[v][u] += path_flow # 순방향에는 +로 흘려줍니다.
                v = parent[v]

        return max_flow # max 반환
    

# Create a graph given in the above diagram


graph1 = [
         [0, 8, 0, 0, 3, 0],
         [0, 0, 9, 0, 0, 0],
         [0, 0, 0, 0, 7, 2],
         [0, 0, 0, 0, 0, 5],
         [0, 0, 7, 4, 0, 0],
         [0, 0, 0, 0, 0, 0]
        ]

graph2 = [
        [0, 16, 13, 0, 0, 0],
        [0, 0, 10, 12, 0, 0],
        [0, 4, 0, 0, 14, 0],
        [0, 0, 9, 0, 0, 20],
        [0, 0, 0, 7, 0, 4],
        [0, 0, 0, 0, 0, 0]
        ]

g1 = Graph(graph1)

g2 = Graph(graph2)



source = 0
sink = 5

print("Graph 1 Max Flow: %d " % g1.ford_fulkerson(source, sink))

print("Graph 2 Max Flow: %d " % g2.ford_fulkerson(source, sink))




Graph 1 Max Flow: 6 
Graph 2 Max Flow: 23 


## Ford-Fulkerson Algorithm Proof of Termination

 
네트워크의 용량이 정수인 경우, augmenting path로 흘려보낼 수 있는 유량은 최소 1이다. 

최대 유량이 F라고 한다면, 최악의 경우에도 F번만큼만 augmenting path를 찾기 때문에 포드 풀커슨 알고리즘은 항상 종료된다. (F의 크기도 유한) 



## Ford-Fulkerson Algorithm 시간 복잡도 증명


Proof of Termination에서 증명했듯, augmenting path를 어떻게 찾느냐에 관계없이 포드 풀커슨 알고리즘은 $O(EF)$안에 종료된다. 

따라서 DFS, BFS로 구현할 때 이 알고리즘은 시간 복잡도가 $O(EF)$이다.  

그런데, BFS로 구현할 때는 $O(EF)$, $O(VE^{2})$ 중 작은 것이라는 시간 복잡도를 갖는다. 

 
1. augmenting path를 찾을 때는 BFS를 사용하므로 $O(V + E) = O(E)$이다.


2. augmenting path는 최대 $O(VE)$번 찾는다. 

이를 다시 증명해야 하는데, 하나의 augmenting path를 찾으면 적어도 한 개의 간선은 포화된다. 

따라서 E개의 간선이 모두 포화되면 종료되는데, 각각의 간선은 최대 V번 만큼 포화될 수 있다. u->v가 포화되었다고 하자. 

그런데 다시 v->u로 유량이 흐른 경우, $dist_{original}(s, v) + 1 \leq dist_{new}(s, v)$가 성립한다. 

그런데 $dist(s, v)$는 V보다 커질 수 없으므로 최대 V번 만큼만 포화될 수 있다. 


 

따라서 $O(E) * O(VE) = O(VE^{2})$이다.


> BFS 를 사용하여 증가경로를 찾는 **Edmonds–Karp 알고리즘** 의 시간 복잡도는 ${\displaystyle O(VE^{2})}$ 입니다.



##  Ford-Fulkerson Algorithm 활용

- 송유관 네트워크에서 두 도시 사이에 보낼 수 있는 석유의 양


- 도로교통망에서 두 도시 사이를 이동할 수 있는 시간당 차량의 수


- 디지털 네트워크에서 두 노드 사이에 전송 가능한 데이터의 대역폭

## [BAEKJOON-6086](https://www.acmicpc.net/problem/6086)

### 문제 

농사꾼 존은 소들이 충분한 물을 마시길 원했다. 

그래서 농장에서 우물에서 외양간을 잇는 N개의 배수관의 지도를 만들기로 했다. 

존은 아주 다양한 크기의 배수관들이 완전히 우연한 방법으로 연결돼있음을 알았다. 


존은 파이프를 통과하는 유량을 계산하고 싶다. 

두개의 배수관이 한줄로 연결 돼 있을 때 두 관의 유량 중 최솟값으로 흐르게 된다.


### 입력 


첫째 줄에 정수 N (1 ≤ N ≤ 700)이 주어진다. 

둘째 줄부터 N+1번째 줄까지 파이프의 정보가 주어진다. 

첫 번째, 두 번째 위치에 파이프의 이름(알파벳 대문자 또는 소문자)이 주어지고, 세 번째 위치에 파이프의 용량이 주어진다.


### 출력

첫째 줄에 A에서 Z까지의 최대 유량을 출력한다.




### Src - 6086

In [2]:
import math
 
sz = 128
 
def bfs(flow, capacity, source, sink):
    
    q = [source]
    parent = [-1] * sz
    parent[source] = source
    
    while len(q) != 0:
        
        item = q.pop(0)
        
        for i in range(sz):
            if parent[i] == -1 and capacity[item][i] - flow[item][i] > 0:
                q.append(i)
                parent[i] = item
        
    return parent
 
    
def maxFlow(capacity, source, sink):
    
    flow = [[0] * sz for _ in range(sz)]
    rst = 0
    
    while True:
        
        parent = bfs(flow, capacity, source, sink)
        
        if parent[sink] == -1:
            return rst
   
        p = sink
        amount = math.inf
        
        while p != source:
            
            amount = min(amount, capacity[parent[p]][p] - flow[parent[p]][p])
            p = parent[p]
        
        rst += amount
        p = sink
        
        while p != source:
            
            flow[parent[p]][p] += amount
            flow[p][parent[p]] -= amount
            p = parent[p]

def solve():
    
    n = int(input())
    capacity = [[0] * sz for _ in range(sz)]
    
    for _ in range(n):
        
        p, q, c = input().split()
        
        p = ord(p)
        q = ord(q)
        c = int(c)
        
        capacity[p][q] += c
        capacity[q][p] += c
    
    print(maxFlow(capacity, ord('A'), ord('Z')))

solve()

5
A B 3
B C 3
C D 5
D Z 4
B Z 6
3
